# Mapping qubits
In this notebook we will cover the QGym `InitialMapping` environment.

This environment is aimed at solving the problem of mapping virtual to physical qubits that have a certain topology.

In [ ]:
%matplotlib inline
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"
import numpy as np
import networkx as nx
from networkx.generators import gnp_random_graph
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.vec_env import SubprocVecEnv
from IPython.display import clear_output

from qgym.envs.initial_mapping import InitialMapping
from qgym.envs.initial_mapping.initial_mapping_rewarders import *

In [ ]:
def render_rgb(step, rgb_array):
    """
    Convenience method that we will use later on to display our results.
    """
    clear_output(wait=True)
    plt.figure(figsize=(40, 20))
    plt.title(f"Step {step}", fontsize=40)
    plt.imshow(rgb_array)
    plt.axis("off")
    plt.show()

### Connection and interaction graph

The initial mapping problem is focussed around two graphs:

- connection graph: hardware layout describing the connections between physical qubits
- interaction graph: software layout describing which virtual qubits interact in the particular quantum program

The goal of the initial mapping problem is to find an optimal one-to-one between the virtual qubits of the interaction graph and the physical qubits of the connection graph.

For now, we will consider an optimal mapping to be any mapping where the number of edges of the mapped interaction graph that do not coincide with edges of the connection graph is minimal.

#### Toy hardware

To explain this concept in more detail we start by defining a toy connection graph and by taking a look at some potential interaction graphs

In [ ]:
connection_graph = nx.Graph()
connection_graph.add_edge(0, 1)
connection_graph.add_edge(0, 2)
connection_graph.add_edge(0, 3)
nx.draw(connection_graph, with_labels=True)

Now let's take a look at some random interaction graphs, and think about how these can be best mapped on the connection graph.

### To Do
Implement the `generate_random_interaction_graph` in the code block below.

_We can simply generate random graphs using [`gnp_random_graph`](https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.gnp_random_graph.html)._

In [ ]:
def generate_random_interaction_graph(connection_graph):
    # todo: implement this
    
interaction_graph = generate_random_interaction_graph(connection_graph)
nx.draw(interaction_graph, with_labels=True)

### `InitialMapping` environment

The `InitialMapping` environment can be initialized by providing:
- a connection graph; and
- an edge probability (i.e. the probability that an edge in the interaction graph exists).

In [ ]:
env = InitialMapping(0.5, connection_graph=connection_graph)

#### State space
The state space is described by a dictionary with the following structure:

- `connection_graph_matrix`: A sparse adjacency matrix of the connection graph.
- `num_nodes`: Number of physical qubits
- `interaction_graph_matrix`: A sparse adjacency matrix of the interaction graph. _(Should have the same number of nodes as the connection graph.)_
- `steps_done`: Number of steps done since the last reset.
- `mapping`: A list of which the index represents a physical qubit and the value a virtual qubit (is set to `num_nodes` + 1 when nothing is mapped to the physical qubit yet).
- `mapping_dict`: A dictionary that maps logical qubits to physical qubit.
- `physical_qubits_mapped`. A set containing all physical qubits that have been mapped already.
- `virtual_qubits_mapped`. A set containing all virtual qubits that have been mapped already.

### To Do
Take a look at the state space in the code block below.

In [ ]:
# todo: take a look at the state space

#### Action space
A valid action is a tuple of integers $(i,j)$, such that $0 \leq i,j < n$ where $n$ is the number of physical qubits. The action $(i,j)$ maps virtual qubit $j$ to phyiscal qubit $i$ when this action is legal. An action is legal when:
1. virtual qubit $i$ has not been mapped to another physical qubit; and
2. no other virual qubit has been mapped to physical qubit $j$.

### To Do
Take a look at the action space in the code block below.

In [ ]:
# todo: take a look at the action space

#### Observation space

Each element in the observation space is a dictionary with 2 entries:
- `mapping`: the current state of the mapping
- `interaction_matrix`: the flattened adjacency matrix of the interaction graph

### To Do
Take a look at the observation space and see what the reset() method returns in the code block below.

In [ ]:
# todo: take a look at the observation space and see what the reset() method returns

#### Rewarders

We have pre-defined 3 different rewarders, all of which return a penalty when an illegal action is attempted, for a valid action their behaviour is as follows:

- `BasicRewarder`: Reward is computed over all edges that have been mapped so far.
- `SingleStepRewader`: Reward is computed over all new edges that have been mapped due to this action.
- `EpisodeRewarder`: Only the final step results in a reward.

All these rewarders can be tweaked by altering either of their parameters:

- `illegal_action_penalty`: penalty given for attempting an illegal action (should be non-positive)
- `reward_per_edge`: reward giving for correctly mapped edges (should be non-negative)
- `penalty_per_edge`: penalty given for incorrectly mapped edges (should be non-positive)

<br/>
<br/>
<br/>
<br/>

### Human Intelligence
Let's attempt to determine the optimal mapping from our randomly generated interaction graph to our toy connection graph. Since this environment is still quite straightforward, we should be able to solve this case optimally by hand.

Don't forget to take a look at the obtained rewards and observations after each step.

_Note: It might be that multiple mappings our optimal._

### To Do
Pick your favorite rewarder in the first code block below.
Next, try to solve the problem by giving the actions which you think are correct. 

In [ ]:
env = InitialMapping(0.5, connection_graph=connection_graph)

# todo: pick your favorite rewarder
# env.rewarder =

obs = env.reset(interaction_graph=interaction_graph)
print(f"observation: {obs}")

In [ ]:
# obs, rewards, done, info = env.step(# todo: fill in the correct action here)
print(f"observation: {obs}\n\nreward: {rewards}")
render_rgb(1, env.render(mode="rgb_array"))

In [ ]:
# obs, rewards, done, info = env.step(# todo: fill in the correct action here)
print(f"observation: {obs}\n\nreward: {rewards}")
render_rgb(2, env.render(mode="rgb_array"))

In [ ]:
# obs, rewards, done, info = env.step(# todo: fill in the correct action here)
print(f"observation: {obs}\n\nreward: {rewards}")
render_rgb(3, env.render(mode="rgb_array"))

In [ ]:
# obs, rewards, done, info = env.step(# todo: fill in the correct action here)
print(f"observation: {obs}\n\nreward: {rewards}")
render_rgb(4, env.render(mode="rgb_array"))

<br/>
<br/>
<br/>
<br/>

### Reinforcement learning

Can we achieve the same using reinforcement learning?

Does changing the rewarder and/or its parameter give better results?

### To Do
Train a model on this environment in the first code block below.
Next, run the second block to see how well the agent performs.

In [ ]:
# todo: train a model on this environment. (1e5 steps should be sufficient)
model.save("initial_mapping_1")

In [ ]:
model = PPO.load("initial_mapping_1")
obs = env.reset(interaction_graph=connection_graph)
for i in range(1000):
    action, states = model.predict(obs, deterministic=False)
    obs, rewards, done, info = env.step(action)
    render_rgb(i, env.render(mode="rgb_array"))
    if done:
        break

Let's try another interaction graph.

### To Do
Design a nice interaction graph in the first code block.
Next, run the second block to see if the agent can map it correctly.

In [ ]:
# todo: make a nice interaction graph

In [ ]:
# test it

Just to be sure, one more...

In [ ]:
# todo: make a nice interaction graph

In [ ]:
# test it

<br/>
<br/>
<br/>
<br/>

### More realistic hardware

Having seen that we are able to train an agent on a toy environment, let's take a look at a more realistic hardware topology.

In [ ]:
connection_graph = nx.Graph()
connection_graph.add_edge(0, 1)
connection_graph.add_edge(1, 2)
connection_graph.add_edge(2, 0)
connection_graph.add_edge(2, 3)
connection_graph.add_edge(3, 4)
connection_graph.add_edge(4, 2)
nx.draw(connection_graph)

### To Do
Use the first code block to create a new environment with the new connection graph, set a rewarder and train an agent.
Use the second code block to design a interaction graph of your choice.
Finally, use the third code block to see how well the agent performs on your interaction graph.

In [ ]:
# todo: train a model on this environment, which rewarder works best? (1e5 steps should be sufficient)
model.save("initial_mapping_2")

In [ ]:
# todo: make a nice interaction graph

In [ ]:
# todo: let's test again (remember that you can load and save different models if you want)

<br/>
<br/>
<br/>
<br/>

### Connection fidelity

Up to now, we have only used the `EpisodeRewarder`. However, we can also define rewarders differently in order to improve performance.

Alternatively, we can define a different rewarder in order to change the objective. Since, for most quantum hardware not every connection has the same fidelity we might want to take this into account for the computation of our reward, such that our agent attempts to find a solution which not only requires a small amount of swap gates but also takes fidelity into account.

Let's first define a weighted connection graph!

### To Do
Define a weighted connection graph in the code block below.

In [ ]:
# todo: define a weighted connection graph

# display graph with edge weights
pos = nx.spring_layout(connection_graph, seed=0)
edge_labels = nx.get_edge_attributes(connection_graph, "weight")
nx.draw(connection_graph, pos, with_labels=True)
nx.draw_networkx_edge_labels(connection_graph, pos, edge_labels);

And now we can implement a new rewarder.

### To Do 
Implement the rest of this rewarder such that it takes the connection fidelity into account

In [ ]:
class FidelityRewarder(BasicRewarder):
    def compute_reward(
        self,
        *,
        old_state: Dict[Any, Any],
        action: NDArray[np.int_],
        new_state: Dict[Any, Any],
    ):
        if self._is_illegal(action, old_state):
            return self._illegal_action_penalty

        # todo: implement the rest of this rewarder such that it takes the connection fidelity into account

Time for training...

In [ ]:
# todo: train and don't forget to SAVE your model

Does this rewarder do the job? Or, is some more tweaking required?

In [ ]:
# todo: which interaction graph would be best to test if our agent has learned what to do?

In [ ]:
# todo: test the models behaviour